# Notebook de desenvolvimento

Foi neste notebook que delimitei o escopo da aplicação e do que veio a se tornar a classe `LLMAccountant`, em `src/llm_application`.

In [15]:
import getpass
import os
import numpy as np
from dotenv import load_dotenv
from uuid import uuid4

from langchain_core.messages import HumanMessage, SystemMessage
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.documents import Document
import pandas as pd

from sklearn.model_selection import train_test_split
from langchain_openai import OpenAIEmbeddings

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import chain
from pydantic import BaseModel, Field
from typing import List, Optional
from langchain_core.tools import tool

In [16]:
load_dotenv(dotenv_path=".env", override=True)

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
df = pd.read_csv("../data/input_contabil.csv")  # Usando aqui a base sem as contas contabeis
df['Conta Contábil'] = 'Inconclusivo'
df['Justificativa LLM'] = ''
df['Status'] = 'Pendente'
df

,Data,Descrição da Transação,Valor,Conta Contábil,Justificativa LLM,Status
0,01/01/2025,Pagamento de aluguel da sede - Janeiro/2025,5500.00,Inconclusivo,,Pendente
1,02/01/2025,Compra de suprimentos de escritório - Papelaria X,230.50,Inconclusivo,,Pendente
2,03/01/2025,PIX recebido de Cliente ABC - Consultoria Fev,15000.00,Inconclusivo,,Pendente
3,04/01/2025,Conta de luz - Eletropaulo - Fatura jan/2025,480.75,Inconclusivo,,Pendente
4,05/01/2025,Salário do mês de Dezembro/2024 - João da Silva,3000.00,Inconclusivo,,Pendente
...,...,...,...,...,...,...
176,26/06/2025,Plano de previdência privada - Bradesco Previd...,300.00,Inconclusivo,,Pendente
177,27/06/2025,Compra de filtro de água - Purificador Plus,180.00,Inconclusivo,,Pendente
178,28/06/2025,Consultoria de SEO - Rankeia Agora,900.00,Inconclusivo,,Pendente
179,29/06/2025,Reparo de fiação elétrica - Eletro Serve,270.00,Inconclusivo,,Pendente


In [ ]:
df_class = pd.read_csv("../data/input_com_categorias.csv")  # versao dos dados com as contas presentes
df_class

,Data,Descrição da Transação,Valor,Conta Contábil
0,01/01/2025,Pagamento de aluguel da sede - Janeiro/2025,5500.00,Despesa - Aluguel
1,02/01/2025,Compra de suprimentos de escritório - Papelaria X,230.50,Despesa - Materiais de Escritório
2,03/01/2025,PIX recebido de Cliente ABC - Consultoria Fev,15000.00,Receita - Serviços Prestados
3,04/01/2025,Conta de luz - Eletropaulo - Fatura jan/2025,480.75,Despesa - Energia Elétrica
4,05/01/2025,Salário do mês de Dezembro/2024 - João da Silva,3000.00,Despesa - Salários
...,...,...,...,...
176,26/06/2025,Plano de previdência privada - Bradesco Previd...,300.00,Despesa - Benefícios a Funcionários
177,27/06/2025,Compra de filtro de água - Purificador Plus,180.00,Despesa - Materiais de Consumo
178,28/06/2025,Consultoria de SEO - Rankeia Agora,900.00,Despesa - Marketing e Publicidade
179,29/06/2025,Reparo de fiação elétrica - Eletro Serve,270.00,Despesa - Manutenção Predial


In [19]:
train_df, test_df = train_test_split(df_class, test_size=0.5, random_state=42)
# Estou injetando 50% da base de dados como conhecimento `a-priori`

### Algo a se salientar aqui é que existe a possibilidade de um `cold-start` nesse _framework_. Podemos simplesmente não realizar a injeção de conhecimento `a-priori`, no entanto, se isso for feito, poderemos visualizar melhor o funcionamento do classificador de LLM.

In [28]:
docs = [Document(page_content=row["Descrição da Transação"]+ "; Valor: R$" + str(row["Valor"]), metadata={
    "category": row["Conta Contábil"],
    "date": row['Data'],
    "value": row['Valor']
    }) for _, row in train_df.iterrows()]

In [ ]:
doc_ids = [str(uuid4()) for _ in range(len(docs))]
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
index = faiss.IndexFlatL2(len(embeddings.embed_query("test")))
source_of_truth = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)
source_of_truth.add_documents(docs, ids=doc_ids, verbose=True)

retriever = source_of_truth.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 10,
        "filter": lambda doc: doc.get('category') != 'Inconclusivo'
    }
)

In [ ]:
known_classes = train_df['Conta Contábil'].unique().tolist()  # lista de contas contabeis validadas pelo ser humano

class ResultClassification(BaseModel):
    category: str = Field(default="Inconclusivo", description="Conta contábil presente na lista de classes conhecidas")
    explanation: str = Field(default='', description="Justificativa plausível para atribuição da conta contábil")
    confidence: float = Field(default=0, description="Nivel de confianca na classificacao", ge=0.0, le=1.0)

class Lancamento(BaseModel):
    date: str = Field(description="Data da transação") #TODO: include the possibility of regular payments
    desc: str = Field(description="Descrição da transação contábil")
    value: float = Field(description="Valor da transação")
    category: ResultClassification = Field(default=None, description="Resultado do modelo de classificacao")
    categorized_by_llm: bool = Field(default=False, description="Indica se a classificacao foi feita por KNN ou auxiliada pelo LLM")
    similar_docs: List[Document] = Field(default = [], description="Documentos similares")
    status: str = Field(default="Pendente", description="Indica se a Classificação já foi validade pelo usuário", enum=["Pendente", "Confirmado", "Alterado"])

In [23]:
def retrieve(query: Lancamento) -> Lancamento:
    retrieve_results = retriever.invoke(query.desc)
    similar_docs = [doc for doc in retrieve_results]
    query.similar_docs = similar_docs
    return query

def decide_classifier(lancamento: Lancamento) -> Lancamento:
    similar_categories = [doc.metadata['category'] for doc in lancamento.similar_docs]
    unq_classes, counts = np.unique(similar_categories, return_counts=True)
    confidence = max(counts)/len(similar_categories)
    if confidence >= 0.8:
        lancamento.categorized_by_llm = False
        lancamento.category = ResultClassification(
            category=unq_classes[np.argmax(counts)],
            explanation='',
            confidence=confidence
        )
        return lancamento
    else:
        lancamento.categorized_by_llm = True
        return lancamento
    
def route_split(lancamento: Lancamento) -> str:
    if lancamento.categorized_by_llm:
        return "llm"
    else:
        return "knn"

def knn_justifier(query: Lancamento) -> Lancamento:
    template = """
    Você é um assistente especializado em justificar a classificação de contas contábeis. 
    Seu papel é fornecer explicações claras e concisas para classificações de contas feitas por modelos classificadores. 
    As descricoes contem pistas (palavras-chave como "aluguel", "boleto", "recebido", "Uber") que associam a transação a uma determianda conta.

    Sua resposta deve seguir exatamente esta estrutura:
    [Justificativa breve usando características da transação e princípios contábeis]

    RESTRIÇÕES:
    - NUNCA inclua saudações, comentários adicionais ou explicações desnecessárias
    - NUNCA questione a classificação fornecida pelo modelo IMPORTANTE!
    - NUNCA use mais de 1 frases na justificativa
    - NUNCA parafraseie o prompt do usuario
    - NUNCA cite o valor da transacao a explicacao, use apenas como referencia
    - SEMPRE a primeira palavra ja deve conter justificativa
    - SEMPRE use terminologia contábil brasileira padrão
    - SEMPRE mantenha foco apenas na justificativa técnica
    - SEMPRE foque em palavras-chave

    EXEMPLOS DE RESPOSTA:
    - A descrição contém 'aluguel' e corresponde a um pagamento periódico de escritório, similar a lançamentos anteriores classificados como Despesa - Aluguel.
    # TODO: add more examples

    User:
    - Gere uma justificativa plausível para atribuição da conta contábil '{category}' 
    para o Lançamento com a descrição '{desc}'.
    """

    prompt = ChatPromptTemplate.from_template(template).invoke({
            'category': query.category.category,
            'desc': query.desc
            }).messages[0]

    explanation = model.invoke(prompt).content
    query.category.explanation = explanation
    return query.category


def llm_classifier(lancamento: Lancamento) -> ResultClassification:
    classifier = model.with_structured_output(ResultClassification)

    prompt_template = """
        Você é um assistente especializado em classificar contas contábeis.
        Seu papel é fornecer classificações precisas e consistentes para lançamentos contábeis com base em descrições de transações fornecidas pelo ususario.

        Esta tarefa inclui as etapas de classificar - justificar - quantificar. 

        justificar:
            - Gere uma explicacao clara e concisa para a classificacao gerada
        quantificar:
            - Gere uma metrica agnostica para a certeza que o assistente possui para a classificacao.
            - Nao seja otimista nem pessimista, a metrica tem que refletir precisamente a quantidade de informacao disponivel.
        classificar:
            - Escolha uma conta contábil da lista de contas contábeis conhecidas. 
            - Se nenhuma conta conhecida se parece com a descricao isto sugere classificacao como 'Inconclusivo'.

        Lista de contas contábeis conhecidas: {known_classes}

        E possivel se guiar pelos meta-dados de lancamentos contabeis similares para aumentar a confianca na classificacao.

        Lista de lancamentos contabeis similares: {similar_documents}

        RESTRIÇÕES NAS METRICAS DE CERTEZA:
        - SEMPRE que a classificacao for 'Inconclusivo'  a metrica de certeza deve ser igual a 0.0

        RESTRIÇÕES NAS JUSTIFICATIVAS:
        - NUNCA inclua saudações, comentários adicionais ou explicações desnecessárias
        - NUNCA questione a classificação fornecida pelo modelo IMPORTANTE!
        - NUNCA use mais de 1 frases na justificativa
        - NUNCA parafraseie o prompt do usuario
        - NUNCA cite o valor da transacao a explicacao, use apenas como referencia
        - SEMPRE a primeira palavra ja deve conter justificativa
        - SEMPRE use terminologia contábil brasileira padrão
        - SEMPRE mantenha foco apenas na justificativa técnica
        - SEMPRE foque em palavras-chave

        EXEMPLOS DE JUSTIFICATIVA:
        - A descrição contém 'aluguel' e corresponde a um pagamento periódico de escritório, similar a lançamentos anteriores classificados como Despesa - Aluguel.
        # TODO: add more examples

        Sua resposta deve seguir exatamente esta estrutura:
        Se classificação == 'Inconclusivo':
        - Justificativa plausivel = ""
        - confianca = 0
        Se classificação != 'Inconclusivo':
        - Justificativa plausivel = [Justificativa breve usando características da transação e princípios contábeis]
        - Confianca = [Valor entre 0 e 1 que reflete a quantidade de informacao existente para embasara a classe atribuida]

        User:
        - Gere uma classificacao - justificativa - quantificacao para a descricao de lancamento contabil a seguir:
        {description}
        """
    
    prompt = ChatPromptTemplate.from_template(prompt_template).invoke(
        {"known_classes": str(known_classes),
        "similar_documents": "\n\n".join(
                                (f"Description: {doc.page_content}\n" f"Metadata: {doc.metadata}")
                                for doc in lancamento.similar_docs
                            ),
        "description": lancamento.desc}
    )

    response = classifier.invoke(prompt)
    if response.confidence >= 0.8 and response.category in known_classes:
        return response
    else:
        return ResultClassification()
    

In [ ]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(Lancamento)  # aqui eu preferi usar o langgraph logo ao inves de conditional chain

workflow.add_node("retrieve", retrieve)
workflow.add_node("decide_classifier", decide_classifier)
workflow.add_node("knn", knn_justifier)
workflow.add_node("llm", llm_classifier)

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "decide_classifier")
workflow.add_conditional_edges(
    "decide_classifier",
    route_split,
    {
        "llm": "llm",
        "knn": "knn"
    }
)
workflow.add_edge("llm", END)
workflow.add_edge("knn", END)

app = workflow.compile()

In [ ]:
test = test_df.iloc[0]
example = Lancamento(
    date=test['Data'],
    desc=test['Descrição da Transação'],
    value=test['Valor'],
)

In [27]:
app.invoke(example)

{'date': '20/01/2025',
 'desc': 'Receita de aluguel de imóvel - Locação sala comercial',
 'value': 2500.0,
 'category': 'Receita - Aluguel',
 'categorized_by_llm': True,
 'similar_docs': [Document(id='08789b2e-d31d-4eec-828a-601eb0c67f52', metadata={'category': 'Receita - Aluguel', 'date': '20/02/2025', 'value': 1000.0}, page_content='Receita de sublocação - Parte do escritório; Valor: R$1000.0'),
  Document(id='914d4e17-520a-4399-afbf-df078f0e284a', metadata={'category': 'Despesa - Aluguel', 'date': '01/06/2025', 'value': 5500.0}, page_content='Aluguel do espaço comercial - Junho/2025; Valor: R$5500.0'),
  Document(id='b1ea7e43-ff11-4ae6-a73e-f7102e917ec7', metadata={'category': 'Receita - Financeiras', 'date': '15/01/2025', 'value': 150.0}, page_content='Receita de juros - Aplicação CDB; Valor: R$150.0'),
  Document(id='0653c687-3680-423a-ab56-aeafb618042c', metadata={'category': 'Despesa - Refeições', 'date': '12/05/2025', 'value': 350.0}, page_content='Reunião almoço com equipe - R

### A partir daqui eu ja tinha o suficiente pra implementar a classe `LLMAccountant`, central para o projeto.